In [1]:
%pip install Augmentor

Note: you may need to restart the kernel to use updated packages.


# 1. Importing modules

In [1]:
import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras.layers import InputLayer
from keras.layers import Dense, Flatten
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, Dropout

from keras.regularizers import l1, l2
from tensorflow.keras.layers import LeakyReLU

# 2. Loading data for a binary classification task

In [43]:
## creating CNN model based on imput size given
def createCNNModel(input_size):
    model = Sequential()

    # First convolutional layer
    model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding='same', input_shape=input_size))
    model.add(Activation('relu'))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # fourth convolutional layer
    model.add(Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Step 3 - Flattening
    model.add(Flatten())

    # Step 4 - Full connection
    model.add(Dense(units = 128, activation = 'relu'))

    model.add(Dense(units = 1, activation = 'sigmoid'))
    model.summary()

    return model

In [44]:
model = createCNNModel((64,64,3))

# Compiling the CNN
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 64, 64, 32)        896       
                                                                 
 activation_12 (Activation)  (None, 64, 64, 32)        0         
                                                                 
 conv2d_13 (Conv2D)          (None, 64, 64, 64)        18496     
                                                                 
 activation_13 (Activation)  (None, 64, 64, 64)        0         
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 32, 32, 64)       0         
 2D)                                                             
                                                                 
 conv2d_14 (Conv2D)          (None, 32, 32, 128)       73856     
                                                      

In [45]:
## run time image pre-processing
train_datagen = ImageDataGenerator(rescale = 1./255,)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [7]:
training_set = train_datagen.flow_from_directory('cats_and_dogs_filtered/train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('cats_and_dogs_filtered/validation',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [36]:
type(training_set)

keras.preprocessing.image.DirectoryIterator

# 3. Basic CNN model without data augmentation

In [8]:
history = model.fit_generator(training_set,
                         epochs = 10,
                         validation_data = test_set,    
                         validation_steps = 2000)

Epoch 1/10


/var/folders/dq/pg0mssxn6yzdwtcdrwnt0bs80000gn/T/ipykernel_13281/1431372863.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(training_set,
2023-07-15 12:21:56.776919: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-07-15 12:21:57.128339: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


63/63 [==============================] - ETA: 0s - loss: 0.7287 - accuracy: 0.5010

2023-07-15 12:22:02.177858: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


63/63 [==============================] - 7s 86ms/step - loss: 0.7287 - accuracy: 0.5010 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/10
63/63 [==============================] - 4s 59ms/step - loss: 0.6901 - accuracy: 0.5080
Epoch 3/10
63/63 [==============================] - 4s 59ms/step - loss: 0.6652 - accuracy: 0.5860
Epoch 4/10
63/63 [==============================] - 4s 59ms/step - loss: 0.6349 - accuracy: 0.6420
Epoch 5/10
63/63 [==============================] - 4s 59ms/step - loss: 0.5916 - accuracy: 0.6860
Epoch 6/10
63/63 [==============================] - 4s 59ms/step - loss: 0.5584 - accuracy: 0.7110
Epoch 7/10
63/63 [==============================] - 4s 59ms/step - loss: 0.5454 - accuracy: 0.7190
Epoch 8/10
63/63 [==============================] - 4s 59ms/step - loss: 0.5404 - accuracy: 0.7260
Epoch 9/10
63/63 [==============================] - 4s 59ms/step - loss: 0.5027 - accuracy: 0.7645
Epoch 10/10
63/63 [==============================] - 4s 59ms/step - loss: 0.46

In [9]:
# evaluate test accuracy
score = model.evaluate(test_set, verbose=0)
accuracy = 100*score[1]

# print test accuracy
print('Test accuracy: %.4f%%' % accuracy)

Test accuracy: 70.1000%


# 4. Now performing data augmentation 

### a. Using Augmentor

In [24]:
import Augmentor
p_train_cats = Augmentor.Pipeline("cats_and_dogs_filtered/train/cats")
p_train_dogs = Augmentor.Pipeline("cats_and_dogs_filtered/train/dogs")
p_test_cats = Augmentor.Pipeline("cats_and_dogs_filtered/validation/cats")
p_test_dogs = Augmentor.Pipeline("cats_and_dogs_filtered/validation/dogs")

Initialised with 1000 image(s) found.
Output directory set to cats_and_dogs_filtered/train/cats/output.Initialised with 1000 image(s) found.
Output directory set to cats_and_dogs_filtered/train/dogs/output.Initialised with 500 image(s) found.
Output directory set to cats_and_dogs_filtered/validation/cats/output.Initialised with 500 image(s) found.
Output directory set to cats_and_dogs_filtered/validation/dogs/output.

In [25]:
p_train_cats.rotate(probability=0.3, max_left_rotation=10, max_right_rotation=10)
p_train_cats.rotate90(probability=0.3)
p_train_cats.rotate270(probability=0.3)

p_train_dogs.rotate(probability=0.3, max_left_rotation=10, max_right_rotation=10)
p_train_dogs.rotate90(probability=0.3)
p_train_dogs.rotate270(probability=0.3)

p_test_cats.rotate(probability=0.3, max_left_rotation=10, max_right_rotation=10)
p_test_cats.rotate90(probability=0.3)
p_test_cats.rotate270(probability=0.3)

p_test_dogs.rotate(probability=0.3, max_left_rotation=10, max_right_rotation=10)
p_test_dogs.rotate90(probability=0.3)
p_test_dogs.rotate270(probability=0.3)

In [26]:
p_train_cats.flip_left_right(probability=0.3)
p_train_cats.flip_top_bottom(probability=0.3)

p_train_dogs.flip_left_right(probability=0.3)
p_train_dogs.flip_top_bottom(probability=0.3)

p_test_cats.flip_left_right(probability=0.3)
p_test_cats.flip_top_bottom(probability=0.3)

p_test_dogs.flip_left_right(probability=0.3)
p_test_dogs.flip_top_bottom(probability=0.3)

In [27]:
p_train_cats.crop_random(probability=.1, percentage_area=0.5)

p_train_dogs.crop_random(probability=.1, percentage_area=0.5)

p_test_cats.crop_random(probability=.1, percentage_area=0.5)

p_test_dogs.crop_random(probability=.1, percentage_area=0.5)

In [28]:
p_train_cats.resize(probability=0.1, width=100, height=100)

p_train_dogs.resize(probability=0.1, width=100, height=100)

p_test_cats.resize(probability=0.1, width=100, height=100)

p_test_dogs.resize(probability=0.1, width=100, height=100)

In [29]:
p_train_cats.random_brightness(probability = 0.5, min_factor=0.4, max_factor=0.9)

p_train_dogs.random_brightness(probability = 0.5, min_factor=0.4, max_factor=0.9)

p_test_cats.random_brightness(probability = 0.5, min_factor=0.4, max_factor=0.9)

p_test_dogs.random_brightness(probability = 0.5, min_factor=0.4, max_factor=0.9)

In [30]:
p_train_cats.random_color(probability=0.5, min_factor=0.4, max_factor=0.9)

p_train_dogs.random_color(probability=0.5, min_factor=0.4, max_factor=0.9)

p_test_cats.random_color(probability=0.5, min_factor=0.4, max_factor=0.9)

p_test_dogs.random_color(probability=0.5, min_factor=0.4, max_factor=0.9)

In [31]:
p_train_cats.random_contrast(probability=0.5, min_factor=0.9, max_factor=1.4)

p_train_dogs.random_contrast(probability=0.5, min_factor=0.9, max_factor=1.4)

p_test_cats.random_contrast(probability=0.5, min_factor=0.9, max_factor=1.4)

p_test_dogs.random_contrast(probability=0.5, min_factor=0.9, max_factor=1.4)

In [32]:
p_train_cats.random_distortion(probability=0.5, grid_width=7, grid_height=8, magnitude=9)

p_train_dogs.random_distortion(probability=0.5, grid_width=7, grid_height=8, magnitude=9)

p_test_cats.random_distortion(probability=0.5, grid_width=7, grid_height=8, magnitude=9)

p_test_dogs.random_distortion(probability=0.5, grid_width=7, grid_height=8, magnitude=9)

In [33]:
p_train_cats.random_erasing(probability=0.5, rectangle_area=0.2)

p_train_dogs.random_erasing(probability=0.5, rectangle_area=0.2)

p_test_cats.random_erasing(probability=0.5, rectangle_area=0.2)

p_test_dogs.random_erasing(probability=0.5, rectangle_area=0.2)

In [34]:
p_train_cats.zoom(probability=0.7, min_factor=1.1, max_factor=1.5)

p_train_dogs.zoom(probability=0.7, min_factor=1.1, max_factor=1.5)

p_test_cats.zoom(probability=0.7, min_factor=1.1, max_factor=1.5)

p_test_dogs.zoom(probability=0.7, min_factor=1.1, max_factor=1.5)

In [35]:
# Change the samples size according to requirements
p_train_cats.sample(1000)

p_train_dogs.sample(1000)

p_test_cats.sample(1000)

p_test_dogs.sample(1000)

Processing <PIL.Image.Image image mode=RGB size=359x360 at 0x15E64BCD0>: 100%|██████████| 1000/1000 [00:03<00:00, 327.41 Samples/s]                 
Processing <PIL.Image.Image image mode=RGB size=247x429 at 0x16A6D2800>: 100%|██████████| 1000/1000 [00:02<00:00, 346.72 Samples/s]
Processing <PIL.Image.Image image mode=RGB size=275x274 at 0x17CCAB460>: 100%|██████████| 1000/1000 [00:03<00:00, 304.35 Samples/s]
Processing <PIL.Image.Image image mode=RGB size=420x380 at 0x15FF48940>: 100%|██████████| 1000/1000 [00:03<00:00, 294.08 Samples/s]


# 5. Loading images from augmentor output

In [42]:
training_set_output= train_datagen.flow_from_directory('cats_and_dogs_filtered/train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')


test_set_output = test_datagen.flow_from_directory('cats_and_dogs_filtered/validation',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')                                      

Found 4000 images belonging to 2 classes.
Found 3000 images belonging to 2 classes.


In [46]:
history = model.fit_generator(training_set,
                         epochs = 10,
                         validation_data = test_set,    
                         validation_steps = 2000)

Epoch 1/10


/var/folders/dq/pg0mssxn6yzdwtcdrwnt0bs80000gn/T/ipykernel_13281/1431372863.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(training_set,
2023-07-15 13:09:58.519876: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


63/63 [==============================] - ETA: 0s - loss: 0.7429 - accuracy: 0.5270

2023-07-15 13:10:02.554683: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


63/63 [==============================] - 5s 78ms/step - loss: 0.7429 - accuracy: 0.5270 - val_loss: 0.6827 - val_accuracy: 0.5010
Epoch 2/10
63/63 [==============================] - 4s 59ms/step - loss: 0.6675 - accuracy: 0.5765
Epoch 3/10
63/63 [==============================] - 4s 59ms/step - loss: 0.6133 - accuracy: 0.6455
Epoch 4/10
63/63 [==============================] - 4s 59ms/step - loss: 0.6003 - accuracy: 0.6790
Epoch 5/10
63/63 [==============================] - 4s 59ms/step - loss: 0.5802 - accuracy: 0.6920
Epoch 6/10
63/63 [==============================] - 4s 59ms/step - loss: 0.5497 - accuracy: 0.7230
Epoch 7/10
63/63 [==============================] - 4s 59ms/step - loss: 0.5176 - accuracy: 0.7275
Epoch 8/10
63/63 [==============================] - 4s 59ms/step - loss: 0.4824 - accuracy: 0.7745
Epoch 9/10
63/63 [==============================] - 4s 59ms/step - loss: 0.4383 - accuracy: 0.8030
Epoch 10/10
63/63 [==============================] - 4s 59ms/step - loss: 0.40

In [47]:
# evaluate test accuracy
score = model.evaluate(test_set, verbose=0)
accuracy = 100*score[1]

# print test accuracy
print('Test accuracy: %.4f%%' % accuracy)

Test accuracy: 68.5000%
